# Original CC 

In [ ]:
#CIFAR-10
import os
import argparse
import torch
import torchvision
import numpy as np
from modules import resnet, network, transform
from evaluation import evaluation
from torch.utils import data
import copy
import gc


In [ ]:
def inference(loader, model, device):
    model.eval()
    feature_vector = []
    labels_vector = []
    j=1
    for step, (x, y) in enumerate(loader):
        x = x.to(device)
        with torch.no_grad():
            c = model.forward_cluster(x)
        c = c.detach()
        feature_vector.extend(c.cpu().detach().numpy())
        if j:
            torch.cuda.empty_cache()
            
            plot_examples(x, y)
            torch.cuda.empty_cache()
            plot_examples(x, c)
            j = 0
        labels_vector.extend(y.numpy())
#         if step % 20 == 0:
#             print(f"Step [{step}/{len(loader)}]\t Computing features...")
    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
#args
# general
seed= 42
workers= 1
dataset_dir= "./dataset"

# train options
batch_size= 256
image_size= 224
start_epoch= 0
epoch= 1000
dataset= "CIFAR-10" # CIFAR-10 / CIFAR-100 / STL-10 / ImageNet-10 / ImageNet-dogs / tiny-ImageNet

# model options
# resnet= "ResNet34" # ResNet18 / ResNet34 / ResNet50
feature_dim= 128
model_path= "save/CIFAR-10"
reload= False

# loss options
learning_rate= 0.0003
weight_decay= 0.
instance_temperature= 0.5
cluster_temperature= 1.0

### CIFAR-10

In [ ]:
image_size = 224
train_dataset = torchvision.datasets.CIFAR10(
    root='./dataset',
    train=True,
    download=True,
    transform=transform.Transforms(size=image_size).test_transform,
)
test_dataset = torchvision.datasets.CIFAR10(
    root='./dataset',
    train=False,
    download=True,
    transform=transform.Transforms(size=image_size).test_transform,
)
dataset = data.ConcatDataset([train_dataset, test_dataset])
class_num = 10

In [ ]:
data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=256,
        shuffle=True,
        drop_last=True,
        num_workers=1,
    )

In [ ]:
def save_examples(images, labels, noise=False, bno=0, adv=False):
    print(type(images[0]), type(labels))
    
    for i in range(min(len(images), 20)):
        img = images[i]
        npimg = img.detach().cpu().numpy()   # convert from tensor
        npimg = np.clip(npimg, 0, 1)
        if not adv:
            plt.imsave(f'Images/C10/orig/CC_c10_b{bno}_{i}_lab{labels[i]}.png', npimg.T, dpi=600)
        if adv:
            plt.imsave(f'Images/C10/adv/CC_c10_b{bno}_{i}_lab{labels[i]}.png', npimg.T, dpi=600)   
        if noise:
            npimg = npimg / 2 + 0.5 
            plt.imsave(f'Images/C10/noise/CC_c10_b{bno}_{i}_noise_lab{labels[i]}.png', npimg.T, dpi=600)
    

In [ ]:
#place the original CC model here
cc_model = "./models/checkpoint_1000.tar"

In [ ]:
res = resnet.get_resnet("ResNet34")
model = network.Network(res, feature_dim, class_num)
# model_fp = os.path.join(model_path, "checkpoint_{}.tar".format(start_epoch))
model.load_state_dict(torch.load(cc_model, map_location=device.type)['net'])
model.to(device)

In [ ]:
print("### Creating features from model ###")
X, Y = inference(data_loader, model, device)

In [ ]:
nmi, ari, f, acc = evaluation.evaluate(Y, X) #Y = Label, X = pred
print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))

## GAN attack

In [ ]:
from gan_attack import GAN_Attack

In [ ]:
use_cuda=True
image_nc=3
epochs = 60
batch_size = 128
BOX_MIN = 0
BOX_MAX = 1
model_num_labels = 10

In [ ]:
GAN = GAN_Attack(device, model, model_num_labels, image_nc, BOX_MIN, BOX_MAX, 'cifar-10')
# model.device


In [ ]:
torch.cuda.empty_cache()

train_dataset = torchvision.datasets.CIFAR10(
    root='./dataset',
    train=True,
    download=True,
    transform=transform.Transforms(size=224, s=0.5),
)
test_dataset = torchvision.datasets.CIFAR10(
    root='./dataset',
    train=False,
    download=True,
    transform=transform.Transforms(size=224, s=0.5),
)

dataset = data.ConcatDataset([train_dataset, test_dataset])

class_num = 10
data_loader2 = torch.utils.data.DataLoader(
    dataset,
    batch_size=128,
    shuffle=True,
    drop_last=True,
    num_workers=1,
)

In [ ]:
import sys

# Holding the original output object. i.e. console out
orig_stdout = sys.stdout

# Opening the file to write file deletion logs.
f = open('outgan_train.txt', 'a+')

# Changing standard out to file out. 
sys.stdout = f
# This will write to the file. 
print("xyz") 

In [ ]:
GAN.train(data_loader2, 120)

In [ ]:
# Closing the file.
f.close()

# replacing the original output format to stdout.
sys.stdout = orig_stdout

# This will print onto the console.
print("xyz") 


## Testing attack

In [ ]:
#Adversarial NMI, ARI, F, and ACC

import models_clu
use_cuda=True
image_nc=3
batch_size = 128



gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/netG_cc_cifar-10_epoch_120.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import pyplot
#For adv inference
def inference2(loader, model, device, clamp=0.03):
    model.eval()
    feature_vector = []
    labels_vector = []
    j = 1
    
    for step, (x, y) in enumerate(loader):
        gc.collect()
        torch.cuda.empty_cache()
        x = x.to(device)
        perturbation = pretrained_G(x)
        
        
        perturbation = torch.clamp(perturbation, -0.15, 0.15)
        adv_img = perturbation + x
        adv_img = torch.clamp(adv_img, 0, 1)
        
        with torch.no_grad():
            c = model.forward_cluster(adv_img)
        c = c.detach()
        
        if j:
            torch.cuda.empty_cache()
            
            plot_examples(x,y)
            torch.cuda.empty_cache()
            plot_examples(adv_img, c)
            j = 0

        feature_vector.extend(c.cpu().detach().numpy())
        labels_vector.extend(y.numpy())
#         if step % 20 == 0:
#             print(f"Step [{step}/{len(loader)}]\t Computing features...")
    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector

In [ ]:
# count_imgs =  1000
def save_examples(images, labels, noise=False, bno=0, adv=False, orig=False):
    print(type(images[0]), type(labels))
    global count_imgs
    
    if count_imgs <=0 :
        return
    for i in range(min(len(images), 20)):
        img = images[i]
        npimg = img.cpu().detach().numpy()   # convert from tensor
        npimg = np.clip(npimg, 0, 1)
        count_imgs -= 1
        if orig:
#             npimg = np.transpose(npimg, (1, 2, 0))
            plt.imsave(f'Images/C10/CC/orig/CC_c10_b{bno}_{i}_lab{labels[i]}.png', npimg.T, dpi=600)
            continue
        if adv:
#             npimg = np.transpose(npimg, (1, 2, 0))
            plt.imsave(f'Images/C10/CC/adv/CC_c10_b{bno}_{i}_lab{labels[i]}.png', npimg.T, dpi=600)
            continue
        if noise:
            npimg = npimg / 2 + 0.5 
            plt.imsave(f'Images/C10/CC/noise/CC_c10_b{bno}_{i}_noise_lab{labels[i]}.png', npimg.T, dpi=600)
            continue
#     plt.show()
    

In [ ]:
for step, (x, y) in enumerate(data_loader):
    gc.collect()
    torch.cuda.empty_cache()
    x = x.to(device)
    perturbation = pretrained_G(x)
    perturbation = torch.clamp(perturbation, -0.15, 0.15)
    adv_x = x + perturbation
    
    perturbation = perturbation.cpu().detach().numpy()
#     print(perturbation[0])
    img = perturbation[30] / 2 + 0.5 
    
    plt.imshow(img.T, cmap="gray")
    adv_x = torch.clamp(adv_x, 0, 1)
    adv_x = adv_x.cpu().detach().numpy()
    
    plt.imshow(adv_x[0].T)
    plt.imshow(x[0].cpu().detach().numpy().T)
    break

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import pyplot
def inference_save(loader, model, device, clamp=0.03):
    model.eval()
    feature_vector = []
    labels_vector = []
    j = 1
    
    for step, (x, y) in enumerate(loader):
        gc.collect()
        torch.cuda.empty_cache()
        x = x.to(device)
        perturbation = pretrained_G(x)
        
        
        perturbation = torch.clamp(perturbation, -0.15, 0.15)
        adv_img = perturbation + x
        adv_img = torch.clamp(adv_img, 0, 1)

        with torch.no_grad():
            c = model.forward_cluster(adv_img)
        c = c.detach()
        
        save_examples(x, y, bno=step, orig=True)
        save_examples(adv_img, c.cpu().detach().numpy(), bno=step, adv=True)
        save_examples(perturbation, c.cpu().detach().numpy(), bno=step, noise=True)
#         raise ValueError
        
        feature_vector.extend(c.cpu().detach().numpy())
        labels_vector.extend(y.numpy())
#         if step % 20 == 0:
#             print(f"Step [{step}/{len(loader)}]\t Computing features...")
    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector

In [ ]:
torch.cuda.empty_cache()
X, Y = inference_save(data_loader, model, 'cuda')

nmi, ari, f, acc = evaluation.evaluate(Y, X)
print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))

## Transferability

In [ ]:
#Base MICE

In [ ]:
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_cifar-10_epoch_120.pth'
pretrained_generator_path = './models/transferability/netG_MICE_cifar-10_epoch_120.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
torch.cuda.empty_cache()
X, Y = inference2(data_loader, model, 'cuda', 0.25)

nmi, ari, f, acc = evaluation.evaluate(Y, X)
print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))

### Base NNM

In [ ]:
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_cifar-10_epoch_120.pth'
pretrained_generator_path = './models/transferability/netG_NNM_cifar-10_epoch_570.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
torch.cuda.empty_cache()
X, Y = inference2(data_loader, model, 'cuda')

nmi, ari, f, acc = evaluation.evaluate(Y, X)
print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))

## Base SCAN

In [ ]:
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_cifar-10_epoch_120.pth'
pretrained_generator_path = './models/transferability/netG_SCAN_cifar-10_epoch_1200.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
torch.cuda.empty_cache()
X, Y = inference2(data_loader, model, 'cuda')

nmi, ari, f, acc = evaluation.evaluate(Y, X)
print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))

## BASE RUC

In [ ]:
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_cifar-10_epoch_120.pth'
pretrained_generator_path = './models/transferability/netG_RUC_cifar-10_epoch600.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

torch.cuda.empty_cache()
X, Y = inference2(data_loader, model, 'cuda')

nmi, ari, f, acc = evaluation.evaluate(Y, X)
print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))

# Base SPICE

In [ ]:
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_cifar-10_epoch_120.pth'
pretrained_generator_path = './models/transferability/netG_SPICE_cifar-10_epoch600.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

torch.cuda.empty_cache()
X, Y = inference2(data_loader, model, 'cuda')

nmi, ari, f, acc = evaluation.evaluate(Y, X)
print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))

# Plot of Pertb Norm vs accuracy

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import pyplot
def inference_n(loader, model, device, clamp=0.05):
    model.eval()
    feature_vector = []
    labels_vector = []
    j = 1
    
    perturb_norm = 0.0
#     p_n = []
    
    for step, (x, y) in enumerate(loader):
        gc.collect()
        torch.cuda.empty_cache()
        x = x.to(device)
        perturbation = pretrained_G(x)
        perturbation = torch.clamp(perturbation, -clamp, clamp)
        perturb_norm += torch.mean(torch.norm(perturbation.view(perturbation.shape[0], -1), 2, dim=1)).to('cpu').item()
        
        
        adv_img = perturbation + x
        adv_img = torch.clamp(adv_img, 0, 1)
        
        with torch.no_grad():
            c = model.forward_cluster(adv_img)

        feature_vector.extend(c.cpu().detach().numpy())
        labels_vector.extend(y.numpy())
    
    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector, perturb_norm/len(loader)

In [ ]:
#Adversarial NMI, ARI, F, and ACC

import models_clu
use_cuda=True
image_nc=3
batch_size = 128



gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/netG_cc_cifar-10_epoch_120.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

norm_l = []
nmi_l = []
ari_l = []
acc_l = []
# clamp = [j for j in range(0, 1, 0.02)]
# clamp = [j for j in np.arange(0, 1.05, 0.05)]
# clamp = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
clamp = [0.0, 0.001, 0.003, 0.005, 0.007, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4]
print(clamp)

for j in clamp:
    torch.cuda.empty_cache()
    X, Y, norm = inference_n(data_loader, model, 'cuda', j) #0.001 -> 10

    print(f'clamp {j} avg norm: {norm}')

    nmi, ari, f, acc = evaluation.evaluate(Y, X)
    print('NMI = {:.4f} ARI = {:.4f} F = {:.4f} ACC = {:.4f}'.format(nmi, ari, f, acc))
    norm_l.append(norm)
    nmi_l.append(nmi)
    ari_l.append(ari)
    acc_l.append(acc)

In [ ]:
# plt.plot(x, y, label = "line 1", linestyle="-")
# plt.plot(y, x, label = "line 2", linestyle="--")
# plt.plot(x, np.sin(x), label = "curve 1", linestyle="-.")
# plt.plot(x, np.cos(x), label = "curve 2", linestyle=":")

plt.plot(norm_l, nmi_l, label = "nmi", linestyle="-")
plt.plot(norm_l, ari_l, label = "ari", linestyle="-")
plt.plot(norm_l, acc_l, label = "acc", linestyle="-")
plt.xlabel("Perturbation Norm")
plt.ylabel("Performace")
plt.legend()
plt.savefig('cc_cifar10.png')
plt.show()

In [ ]:
print(norm_l)
print()
print(nmi_l)
print()
print(ari_l)
print()
print(acc_l)

In [ ]:
#Same eval metrics across all codes

In [ ]:
import eval_cus

In [ ]:
#Adversarial NMI, ARI, F, and ACC

import models_clu
use_cuda=True
image_nc=3
batch_size = 128
class_names = ('plane', 'car', 'bird', 'cat',
       'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/netG_cc_cifar-10_epoch_120.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

norm_l = []
nmi_l = []
ari_l = []
acc_l = []
# clamp = [j for j in range(0, 1, 0.02)]
# clamp = [j for j in np.arange(0, 1.05, 0.05)]
# clamp = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
clamp = [0.0, 0.001, 0.003, 0.005, 0.007, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4]
print(clamp)

for j in clamp:
    torch.cuda.empty_cache()
    X, Y, norm = inference_n(data_loader, model, 'cuda', j)
    Y = torch.from_numpy(Y).cuda()    
    X = torch.from_numpy(X).cuda()
    
    clustering_stats_adv = eval_cus.check(Y, X, 10, class_names, compute_confusion_matrix=True, confusion_matrix_file=None)
    acc = clustering_stats_adv['ACC']
    nmi = clustering_stats_adv['NMI']
    ari = clustering_stats_adv['ARI']
    print(f'clamp {j} avg norm: {norm}')

#     nmi, ari, f, acc = evaluation.evaluate(Y, X)
    print('NMI = {:.4f} ARI = {:.4f} ACC = {:.4f}'.format(nmi, ari, acc))
    norm_l.append(norm)
    nmi_l.append(nmi)
    ari_l.append(ari)
    acc_l.append(acc)

In [ ]:
print(norm_l)
print()
print(nmi_l)
print()
print(ari_l)
print()
print(acc_l)

In [ ]:
#Adversarial NMI, ARI, F, and ACC

import models_clu
use_cuda=True
image_nc=3
batch_size = 128
class_names = ('plane', 'car', 'bird', 'cat',
       'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/netG_cc_cifar-10_epoch_120.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

norm_l = []
nmi_l = []
ari_l = []
acc_l = []
# clamp = [j for j in range(0, 1, 0.02)]
# clamp = [j for j in np.arange(0, 1.05, 0.05)]
# clamp = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
# clamp = [0.0, 0.001, 0.003, 0.005, 0.007, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4]
clamp = [0.03]
print(clamp)

for j in clamp:
    torch.cuda.empty_cache()
    X, Y, norm = inference_n(data_loader, model, 'cuda', j)
    Y = torch.from_numpy(Y).cuda()    
    X = torch.from_numpy(X).cuda()
    
    clustering_stats_adv = eval_cus.check(Y, X, 10, class_names, compute_confusion_matrix=True, confusion_matrix_file=None)
    acc = clustering_stats_adv['ACC']
    nmi = clustering_stats_adv['NMI']
    ari = clustering_stats_adv['ARI']
    print(f'clamp {j} avg norm: {norm}')

#     nmi, ari, f, acc = evaluation.evaluate(Y, X)
    print('NMI = {:.4f} ARI = {:.4f} ACC = {:.4f}'.format(nmi, ari, acc))
    norm_l.append(norm)
    nmi_l.append(nmi)
    ari_l.append(ari)
    acc_l.append(acc)

In [ ]:
#epochs:
import eval_cus
# Queries

In [ ]:
NMI_l = []
ARI_l = []
ACC_l = []
Epo = []
norm_l = [] 
class_names = ('airplane', 'car', 'bird', 'cat',
       'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
for epo in range(10,91, 10):
    gc.collect()
    torch.cuda.empty_cache()

    pretrained_generator_path = f'./models/netG_cc_test_cifar-10_epoch_{epo}.pth'
    pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
    pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
    pretrained_G.eval()
    torch.cuda.empty_cache()
    X, Y, norm = inference_n(data_loader, model, 'cuda', 0.03)
    
    del pretrained_G
    Y = torch.from_numpy(Y).cuda()    
    X = torch.from_numpy(X).cuda()
    
    clustering_stats_adv = eval_cus.check(Y, X, 10, class_names, compute_confusion_matrix=True, confusion_matrix_file=None)
    acc = clustering_stats_adv['ACC']
    nmi = clustering_stats_adv['NMI']
    ari = clustering_stats_adv['ARI']
    print(f'clamp {epo} avg norm: {norm}')

#     nmi, ari, f, acc = evaluation.evaluate(Y, X)
    print('NMI = {:.4f} ARI = {:.4f} ACC = {:.4f}'.format(nmi, ari, acc))
    
    norm_l.append(norm)
    NMI_l.append(nmi)
    ARI_l.append(ari)
    ACC_l.append(acc)
    Epo.append(epo*468) #468 queries in one epoch.
    

In [ ]:
print(NMI_l)
print(ARI_l)
print(ACC_l)
print(Epo)

In [ ]:
plt.plot(Epo, NMI_l, label = "nmi", linestyle="-")
plt.plot(Epo, ARI_l, label = "ari", linestyle="-")
plt.plot(Epo, ACC_l, label = "acc", linestyle="-")
plt.xlabel("Queries")
plt.ylabel("Performace")
plt.legend()
plt.grid()
plt.savefig('scan_cifar10.png')
plt.show()

In [ ]:
plt.plot(Epo, norm_l, label = "norm", linestyle="-")
plt.xlabel("Queries")
plt.ylabel("Performace")
plt.legend()
plt.grid()
plt.savefig('scan_cifar10.png')
plt.show()

In [ ]:
import eval_cus

In [ ]:
#Adversarial NMI, ARI, F, and ACC

import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/netG_cc_cifar-10_epoch_120.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
clamp = [0.0, 0.03, 1]
print(clamp)
class_names = ('airplane', 'car', 'bird', 'cat',
       'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
for j in clamp:
    torch.cuda.empty_cache()
    X, Y, norm = inference_n(data_loader, model, 'cuda', j)
    Y = torch.from_numpy(Y).cuda()    
    X = torch.from_numpy(X).cuda()
    
    clustering_stats_adv = eval_cus.check(Y, X, 10, class_names, confusion_matrix_file=None, cf20=False, output_file2=f'CC_c10_{j}_n{norm}.pdf')
    acc = clustering_stats_adv['ACC']
    nmi = clustering_stats_adv['NMI']
    ari = clustering_stats_adv['ARI']
    print(f'clamp {j} avg norm: {norm}')

#     nmi, ari, f, acc = evaluation.evaluate(Y, X)
    print('NMI = {:.4f} ARI = {:.4f} ACC = {:.4f}'.format(nmi, ari, acc))
    print(norm)
#     nmi_l.append(nmi)
#     ari_l.append(ari)
#     acc_l.append(acc)